# Text document Classification (Dataset)

https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification

*   10 clases
*   1000 muestras de entrenamiento

In [1]:
import zipfile
with zipfile.ZipFile('/content/Dataset_text.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Dataset_text/')

In [2]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "/content/Dataset_text",
    batch_size=batch_size,
    validation_split=0.2,
    label_mode="categorical",
    seed = 2023,
    subset="training",
)

Found 1000 files belonging to 10 classes.
Using 800 files for training.


In [3]:
val_ds = keras.utils.text_dataset_from_directory(
    "/content/Dataset_text",
    batch_size=batch_size,
    validation_split=0.2,
    label_mode="categorical",
    seed = 2023,
    subset="validation",
)

Found 1000 files belonging to 10 classes.
Using 200 files for validation.


# Data preparation



In [4]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# No se utilizan ngrams (none)
max_length = 600
max_tokens = 20000
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,    # Limitar la entrada a 600 palabras
)

# Extraer solo el texto (features) de los datos de entrada para calcular el vocabulario
text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

# Modelo

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Dropout, Bidirectional


inputs = Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)

x = Bidirectional(layers.LSTM(32))(embedded)
x = Dropout(0.5)(x)
outputs = Dense(10, activation="softmax")(x)

model_LSTM = keras.Model(inputs, outputs)

model_LSTM.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model_LSTM.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirection  (None, 64)                5128448   
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 10)                650       
                                                                 
Total params: 5129098 (19.57 MB)
Trainable params: 5129098 (19.57 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

# Entrenamiento

In [6]:
# Utilización de caché
model_LSTM.fit(
    int_train_ds,
    validation_data=int_val_ds,
    epochs=20,
    #callbacks=callbacks
    )

Epoch 1/20
25/25 [==============================] - 13s 289ms/step - loss: 2.2987 - accuracy: 0.1300 - val_loss: 2.2928 - val_accuracy: 0.1850
Epoch 2/20
25/25 [==============================] - 6s 233ms/step - loss: 2.2663 - accuracy: 0.1838 - val_loss: 2.2220 - val_accuracy: 0.2050
Epoch 3/20
25/25 [==============================] - 6s 231ms/step - loss: 2.1551 - accuracy: 0.1950 - val_loss: 2.1117 - val_accuracy: 0.1900
Epoch 4/20
25/25 [==============================] - 6s 257ms/step - loss: 2.0477 - accuracy: 0.2188 - val_loss: 2.0787 - val_accuracy: 0.1850
Epoch 5/20
25/25 [==============================] - 6s 233ms/step - loss: 1.9250 - accuracy: 0.2725 - val_loss: 2.0132 - val_accuracy: 0.2000
Epoch 6/20
25/25 [==============================] - 6s 237ms/step - loss: 1.8461 - accuracy: 0.2688 - val_loss: 1.8704 - val_accuracy: 0.2450
Epoch 7/20
25/25 [==============================] - 6s 234ms/step - loss: 1.7703 - accuracy: 0.3075 - val_loss: 1.7521 - val_accuracy: 0.2850
Epoch

#Evaluación del modelo

Sobre datos test